In [6]:
#installing libs
%pip install praw pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
#importing required libs for scraping
import praw
import pandas as pd
from praw.models import MoreComments
import time
from datetime import datetime, timezone

In [2]:
#reddit creds
reddit = praw.Reddit(
    user_agent="Comment Extraction (by /u/lestergreeks)",
    client_id="YiK4KHJXxneFv0IiV8aOhg", 
    client_secret="8GfGLUx8E62B3sBUwJcie43RDBQm7A"
)

In [3]:
#using a reddit post
#https://www.reddit.com/r/IndiaSpeaks/comments/1fr7tyb/scrap_dealer_gifts_multiple_iphones_to_son_for/ - apple test
url = "https://www.reddit.com/r/IndiaSpeaks/comments/1fqhbsz/youtuber_ranveer_allahbadias_youtube_channel_got/"
submission = reddit.submission(url=url)

In [4]:
#getting all comments from a post
submission.comments.replace_more(limit=None)
posts = []
for top_level_comment in submission.comments.list():
    if isinstance(top_level_comment, MoreComments):
        continue
    if top_level_comment.author and "bot" not in top_level_comment.author.name.lower() and not top_level_comment.stickied:
        posts.append(top_level_comment.body)
posts = pd.DataFrame(posts,columns=["body"])
indexNames = posts[(posts.body == '[removed]') | (posts.body == '[deleted]')].index
posts.drop(indexNames, inplace=True)

In [5]:
posts

,body
0,I heard both his passwords where 1234567@ 💀
1,Hogya recover
2,It was the hackers who showed him the next doo...
3,"What this basically tells is that:\n\n""Life is..."
4,This is the exact reason why India should deve...
...,...
68,YouTube has decades worth of data plus backed ...
69,Yes they can join. But Youtube algo recommends...
70,1. This is completely unrelated to Ranbir's ch...
71,Lmao yoh are placing to much importance on tec...


In [8]:
#extracting all comments from top10 posts of a subreddit
subreddit = reddit.subreddit('UpliftingNews')

all_comments = pd.DataFrame()


def extract_comments(submission):
    posts = []
    submission.comments.replace_more(limit=None)
    
    for top_level_comment in submission.comments.list():
        if isinstance(top_level_comment, MoreComments):
            continue
        if (top_level_comment.author and "bot" not in top_level_comment.author.name.lower() 
            and not top_level_comment.stickied):
            posts.append(top_level_comment.body)
    
    return posts

def process_top_posts():
    global all_comments
    attempt = 0
    for submission in subreddit.top(limit=10, time_filter='all'):
        try:
            print(f"Processing post: {submission.title}")
            posts = extract_comments(submission)
            posts_df = pd.DataFrame(posts, columns=["body"])
            indexNames = posts_df[(posts_df.body == '[removed]') | (posts_df.body == '[deleted]')].index
            posts_df.drop(indexNames, inplace=True)
            posts_df['post_title'] = submission.title
            posts_df['post_time'] = datetime.fromtimestamp(submission.created_utc, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

            all_comments = pd.concat([all_comments, posts_df], ignore_index=True)
        
        except praw.exceptions.RedditAPIException as e:
            if "RATELIMIT" in str(e):
                attempt += 1
                sleep_time = 120 * attempt
                print(f"Rate limit hit. Sleeping for {sleep_time} seconds. Error: {e}")
                time.sleep(sleep_time) 
                continue
            else:
                print(f"Encountered an error: {e}")
                continue
    
        time.sleep(60)


process_top_posts()
    

Processing post: Billionaire Mark Cuban just launched a drug company that is “dedicated to producing low-cost versions of high-cost generic drugs” and pledges “radical transparency” in pricing.
Processing post: Man falsely imprisoned for 10 years, uses prison library to study law and have his conviction overturned. Becomes a lawyer and is now helping overturn other false convictions.
Processing post: First paralyzed human treated with stem cells has now regained his upper body movement.
Processing post: Over a Million People Sign Petition Calling For KKK to Be Declared a Terrorist Group
Processing post: Hollywood Superstar Keanu Reeves Has Secretly Been Financing Countless Children’s Hospitals
Processing post: Amazon tribe wins legal battle against oil companies. Preventing drilling in Amazon Rainforest
Processing post: Belgium approves four-day week and gives employees the right to ignore their bosses after work
Processing post: Billionaire No More: Patagonia Founder Gives Away the Co

In [9]:
all_comments

,body,post_title,post_time
0,"Insulin, insulin, insulin, insulin.",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
1,The brand name drug my doctor prescribed me is...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
2,The fact that insulin isn't practically free i...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
3,You don’t need to ‘produce’ the drugs. You jus...,Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
4,"Call me a pessimist, but I'll believe it when ...",Billionaire Mark Cuban just launched a drug co...,2021-01-27 04:00:57
...,...,...,...
21228,Now try to “change it”,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21229,Why would I try to pass a law prohibiting murd...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21230,Jesus Christ you are dense. Yes a law prohibit...,Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27
21231,"Damn right I'm being dense, because you are to...",Chattanooga's Police Chief has updated his dep...,2020-06-09 14:45:27


In [10]:
all_comments.to_csv('red_data.csv')